# DATA PREPARATION 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Importing research data
https://github.com/WisnuHanif/reactor_data/blob/main/reactor_data.csv'
prep0 = pd.read_csv('https://raw.githubusercontent.com/WisnuHanif/reactor_data/main/reactor_data.csv')
prep0.head()

In [ ]:
#Identity variables name
prep0.columns

In [ ]:
#Removing variable description & 'NO', 'Time' column
prep1 = prep0.iloc[1:, :].drop(['Running_cycle','Time'], axis=1)
prep1.head()

In [ ]:
#Convert timestamp object data to numerical
prep2 = prep1.apply(pd.to_numeric)
print(prep2.dtypes, prep2.shape)

In [ ]:
#Check if there's missing value
prep2.isnull().sum()

In [ ]:
#Removing data where plant is not run, by identifying total raw material 'FI-001' loss flow rate
import seaborn as sns
sns.boxplot(data=prep2,x=prep2['FI-001'])

In [ ]:
#Remove shut down data by identifying outlier FI-001 with Inter Quantile Range Method

from numpy import percentile
# calculate interquartile range
q25_a, q75_a = percentile(prep2['FI-001'], 25), percentile(prep2['FI-001'], 75)
iqr_a = q75_a - q25_a
print('Percentiles: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25_a, q75_a, iqr_a))
# calculate the outlier cutoff
cut_off_a = iqr_a * 1.5
lower_a, upper_a = q25_a - cut_off_a, q75_a + cut_off_a
print('Lower whisker=%.2f, Upper whisker=%.2f' % (lower_a, upper_a))
# identify outliers
shut_down_data = prep2[(prep2['FI-001']<lower_a)|(prep2['FI-001']>upper_a)]
print('Shut down data: %d' % len(shut_down_data))
# remove outliers
shut_down_removed = prep2[(prep2['FI-001']>lower_a)&(prep2['FI-001']<upper_a)]
print('Non-Shut down data: %d' % len(shut_down_removed))

In [ ]:
#Check again if there's still outlier in 'FI-001'
sns.boxplot(data=shut_down_removed, x=shut_down_removed['FI-001'])

In [ ]:
prep3 = shut_down_removed
prep3.shape

In [ ]:
#Remove outlier for all variables while keeps the whole row intact
lb = prep3.quantile(0.01)
ub = prep3.quantile(0.99)

prep4 = prep3[(prep3 > lb) & (prep3 < ub)]
prep4

In [ ]:
prep4.info()

In [ ]:
#Check deleted value position
import missingno as mno
mno.matrix(prep4, figsize = (20, 6))

In [ ]:
#Correlation matrix between variables before missing value imputation
#corr = prep4.corr()
#corr.style.background_gradient(cmap='coolwarm')

In [ ]:
#corr.values[np.triu_indices_from(corr.values,1)].sum()

In [ ]:
#Fill missing value (from removed outlier) with imputer
prep5 = prep4.interpolate(method ='linear', limit_direction ='forward')
prep5.head()

In [ ]:
prep5.isnull().sum()

In [ ]:
prep5.describe().transpose()

In [ ]:
#Correlation matrix after data imputation
corr2 = prep5.corr()
corr2.style.background_gradient(cmap='coolwarm')

In [ ]:
#Visualization plot for all variables
#group_1 = prep5.iloc[:,0:9]
#group_2 = prep5.iloc[:,9:18]
#group_3 = prep5.iloc[:,18:27]
#group_4 = prep5.iloc[:,27:36]
#group_5 = prep5.iloc[:,36:45]
#group_6 = prep5.iloc[:,45:54]
#group_7 = prep5.iloc[:,54:64]

In [ ]:
#Plot for group 1
#group_1.plot(subplots =True, sharex = True, figsize = (30,80))
#Plot for group 2
#group_2.plot(subplots =True, sharex = True, figsize = (30,80))
#Plot for group 3
#group_3.plot(subplots =True, sharex = True, figsize = (30,80))
#Plot for group 4
#group_4.plot(subplots =True, sharex = True, figsize = (30,80))
#Plot for group 5
#group_5.plot(subplots =True, sharex = True, figsize = (30,80))
#Plot for group 6
#group_6.plot(subplots =True, sharex = True, figsize = (30,80))
#Plot for group 7
#group_7.plot(subplots =True, sharex = True, figsize = (40,40))

In [ ]:
# Pearson correlation coefficient
select_corr = prep5.corr()["CONVERSION"].sort_values(ascending=False)[1:]

# absolute for positive values
abs_corr = abs(select_corr)

# random threshold for features to keep
selected_features = abs_corr[abs_corr>0.4]
len(selected_features)

In [ ]:
# Drop low correlation features
prep6 = prep5[selected_features.index].interpolate(method ='linear', limit_direction ='backward')
prep6.isnull().sum()

In [ ]:
y = prep5["CONVERSION"]
X = prep6

In [ ]:
#remove collinearity by removing irrelavant features with ebbedded method
from sklearn.linear_model import LassoCV
reg = LassoCV()
reg.fit(X, y)
print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(X,y))
coef = pd.Series(reg.coef_, index = X.columns)

In [ ]:
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

In [ ]:
#Visualize important feature
imp_coef = coef.sort_values()
import matplotlib
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Feature importance using Lasso Model")

In [ ]:
#Select relevant reatures
abs_coef = abs(coef)
relevant_features = abs_coef[abs_coef>0]
prep7 = prep6[relevant_features.index].sort_index(axis=1, ascending=True)
prep7['CONVERSION'] = prep5["CONVERSION"]
prep7.shape

In [ ]:
corr3 = prep7.corr().style.background_gradient(cmap='coolwarm')
corr3

In [ ]:
#Data Scaling with normalization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# transform data
scaled_data = pd.DataFrame(scaler.fit_transform(prep7), columns = prep7.columns)
print(scaled_data)

In [ ]:
data = prep7

# MACHINE LEARNING MODEL : RANDOM FORREST REGRESSION

In [ ]:
rfr_data = data
#Import the features
features = data.drop('CONVERSION', axis = 1)
# Extract the labels
targets = data['CONVERSION']

In [ ]:
feature_list = list(features.columns)
feature_names = features.columns

In [ ]:
print(features.shape, targets.shape)

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

#Create split data for targets
train_targets_size = int(len(targets) * 0.8)
train_targets, test_targets = targets[0:train_targets_size], targets[train_targets_size:len(targets)]
print('Observations: %d' % (len(targets)))
print('Training Observations: %d' % (len(train_targets)))
print('Testing Observations: %d' % (len(test_targets)))
plt.plot(train_targets)
plt.plot([None for i in train_targets] + [x for x in test_targets])
plt.show()

In [ ]:
#Create split data for features
train_features_size = int(len(features) * 0.8)
train_features, test_features = features[0:train_features_size], features[train_features_size:len(features)]
print('Observations: %d' % (len(features)))
print('Training Observations: %d' % (len(train_features)))
print('Testing Observations: %d' % (len(test_features)))
#plt.plot(train_features)
#plt.plot([None for i in train_features] + [x for x in test_features])
#plt.show()

In [ ]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_targets.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_targets.shape)

In [ ]:
# Imputation of missing values
train_features = train_features.fillna(train_features.mean())
test_features = test_features.fillna(test_features.mean())

In [ ]:
#Run Ramdom Forest Regressor without hyper parameter tuning (default)
%timeit
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 100, oob_score = True)
regressor.fit(train_features, train_targets)

In [ ]:
#Generate Regressor score and OOB Score of the model
print("\nRegressor Score " + str(regressor.score(train_features, train_targets)), "\nOOB Score " + str(regressor.oob_score_))

In [ ]:
n_nodes = []
max_depths = []

# Stats about the trees in random forest
for ind_tree in regressor.estimators_:
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

In [ ]:
y_pred = regressor.predict(test_features)
y_pred

In [ ]:
# evaluate predictions
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score 

print('Mean Absolute Error:', metrics.mean_absolute_error(test_targets, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(test_targets, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(test_targets, y_pred)))
print('Coefficient of Determination:', r2_score(test_targets, y_pred)) 
print('Accuracy:', 100 - (100 * np.mean((abs(y_pred - test_targets)) / test_targets)))

In [ ]:
test_targets = test_targets.values

In [ ]:
#Plot actual vs prediction
with plt.style.context('ggplot'):
    plt.figure()
    plt.plot(test_targets, label = "Actual Conversion")
    plt.plot(y_pred, label = "Prediction")
    plt.title('Random Forrest Regression Prediction')
    plt.xlabel('Time')
    plt.ylabel('Conversion (%)')
    plt.legend(loc='best')
    plt.show()

In [ ]:
# Calculate the absolute errors
errors = abs(y_pred - test_targets)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

In [ ]:
fi = pd.DataFrame({'feature': feature_list,
                   'importance': regressor.feature_importances_}).\
                    sort_values('importance', ascending = False)
fi

In [ ]:
#Random Search with Cross Validation
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 100, num = 19)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = 100, scoring='neg_mean_absolute_error', 
                              cv = 4, verbose=2, n_jobs=-1,
                              return_train_score=True)

# Fit the random search model
rf_random.fit(train_features, train_targets);

In [ ]:
rf_random.best_params_

In [ ]:
rf_random.cv_results_

In [ ]:
rf_random.best_estimator_

In [ ]:
best_random = rf_random.best_estimator_
best_pred = best_random.predict(test_features)
best_pred

In [ ]:
#Evaluate the Best Random Search Model

print('Mean Absolute Error:', metrics.mean_absolute_error(test_targets, best_pred))
print('Mean Squared Error:', metrics.mean_squared_error(test_targets, best_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(test_targets, best_pred)))
print('Coefficient of Determination:', r2_score(test_targets, best_pred)) 
print('Accuracy:', 100 - (100 * np.mean((abs(best_pred - test_targets)) / test_targets)))

In [ ]:
#Training Curves
from sklearn.model_selection import GridSearchCV
tree_grid = {'n_estimators': [int(x) for x in np.linspace(1, 301, 30)]}

# Create the grid search model and fit to the training data
tree_grid_search = GridSearchCV(best_random, param_grid=tree_grid, verbose = 2, n_jobs=-1, cv = 4,
                                scoring = 'neg_mean_absolute_error')
tree_grid_search.fit(train_features, train_targets)

In [ ]:
#Plot actual vs prediction
with plt.style.context('ggplot'):
    plt.figure()
    plt.plot(test_targets, label = "Actual Conversion")
    plt.plot(best_pred, label = "Prediction")
    plt.title('Random Forrest Regression Prediction (Best Parameter Tuning)')
    plt.xlabel('Time')
    plt.ylabel('Conversion (%)')
    plt.legend(loc='best')
    plt.show()

In [ ]:
train_scores = rf_random.cv_results_['mean_train_score']
len(train_scores)

In [ ]:
def plot_results(model, param = 'n_estimators', name = 'Num Trees'):
    param_name = 'param_%s' % param

    # Extract information from the cross validation model
    #train_scores = model.cv_results_['mean_train_score']
    test_scores = model.cv_results_['mean_test_score']
    train_time = model.cv_results_['mean_fit_time']
    param_values = list(model.cv_results_[param_name])
    
    # Plot the scores over the parameter
    plt.subplots(1, 2, figsize=(10, 6))
    plt.subplot(121)
    #plt.plot(param_values, train_scores, 'bo-', label = 'train')
    plt.plot(param_values, test_scores, 'go-', label = 'test')
    plt.ylim(ymin = -2.5, ymax = 0)
    plt.legend()
    plt.grid()
    plt.xlabel(name)
    plt.ylabel('Neg Mean Absolute Error')
    plt.title('Score vs %s' % name)
    
    plt.subplot(122)
    plt.plot(param_values, train_time, 'ro-')
    plt.ylim(ymin = 0.0, ymax = 40.0)
    plt.grid()
    plt.xlabel(name)
    plt.ylabel('Train Time (sec)')
    plt.title('Training Time vs %s' % name)
    
    
    plt.tight_layout(pad = 4)

In [ ]:
plot_results(tree_grid_search)

# MACHINE LEARNING MODEL : SUPPORT VECTOR REGRESSION

In [ ]:
#1 Select scaled data
svr_data = scaled_data

In [ ]:
#2. import data for SVR

x = svr_data.drop('CONVERSION', axis = 1).values.astype(float).reshape(-1, 16)
y = svr_data['CONVERSION'].values.astype(float)
print(x.shape, y.shape)

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

#Create split data for targets
y_train_size = int(len(y) * 0.8)
y_train, y_test = y[0:y_train_size], y[y_train_size:len(y)]
print('Observations: %d' % (len(y)))
print('Training Observations: %d' % (len(y_train)))
print('Testing Observations: %d' % (len(y_test)))
plt.plot(y_train)
plt.plot([None for i in y_train] + [x for x in y_test])
plt.show()

In [ ]:
#Create split data for features
x_train_size = int(len(x) * 0.8)
x_train, x_test = x[0:x_train_size], x[x_train_size:len(x)]
print('Observations: %d' % (len(x)))
print('Training Observations: %d' % (len(x_train)))
print('Testing Observations: %d' % (len(x_test)))

In [ ]:
print('Training Features Shape:', x_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', x_test.shape)
print('Testing Labels Shape:', y_test.shape)

In [ ]:
from sklearn.svm import SVR
svr_rbf = SVR(kernel = 'rbf')
svr_rbf.fit(x_train, y_train)

In [ ]:
svr_pred = svr_rbf.predict(x_test)
svr_pred

In [ ]:
#Evaluating SVR performance
from sklearn import metrics
from sklearn.metrics import r2_score 

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, svr_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, svr_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, svr_pred)))
print('Coefficient of Determination:', r2_score(y_test, svr_pred)) 

In [ ]:
#Plot actual vs prediction
with plt.style.context('ggplot'):
    plt.figure()
    plt.plot(y_test, label = "Actual Conversion")
    plt.plot(svr_pred, label = "Prediction")
    plt.title('Support Vector Regression Prediction')
    plt.xlabel('Time')
    plt.ylabel('Conversion (%)')
    plt.legend(loc='best')
    plt.show()

In [ ]:
#SVR polynomial kernel with 3 degree
svr_poly3 = SVR(kernel='poly', gamma='auto', degree=3)
#SVR polynomial kernel with 4 degree
svr_poly4 = SVR(kernel='poly', gamma='auto', degree=4)
#SVR polynomial kernel with 5 degree
svr_poly5 = SVR(kernel='poly', gamma='auto', degree=5)

In [ ]:
svrs = [svr_rbf, svr_poly3, svr_poly4, svr_poly5]
kernel_label = ['rbf', '3 degree Polynomial', '4 degree Polynomial', '5 degree Polynomial']
fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(10, 10), sharey=True)
for ix, svr in enumerate(svrs):
    svr.fit(x_train, y_train)
    svr_poly_pred = svr.predict(x_test)
    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, svr_poly_pred))
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, svr_poly_pred))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, svr_poly_pred)))
    print('Coefficient of Determination:', r2_score(y_test, svr_poly_pred)) 
    with plt.style.context('ggplot'):
        axes[ix].plot(y_test, label = "Actual Conversion")
        axes[ix].plot(svr_poly_pred,
                  label='{} model'.format(kernel_label[ix]))
        axes[ix].legend(loc='upper center', bbox_to_anchor=(0.5, 1.1),
                    ncol=1, fancybox=True, shadow=True)
        axes[ix].legend(loc='best')
fig.text(0.5, 0.04, 'Time', ha='center', va='center')
fig.text(0.06, 0.5, 'Conversion', ha='center', va='center', rotation='vertical')
fig.suptitle("Support Vector Regression Model", fontsize=14)
plt.show()

In [ ]:
#Set parameter for grid search
kernel = ['linear', 'poly', 'rbf', 'sigmoid']
gamma = ['scale', 'auto']
C = [0.1, 1, 10, 100, 1000]
epsilon = [0.1, 0.2, 0.3, 0.4, 0.5]
degree = [2, 3, 4, 5]

# Create the random grid
param_grid = {'kernel': kernel,
               'gamma': gamma,
               'C': C,
               'epsilon': epsilon,
               'degree': degree}

print(param_grid)

In [ ]:
#Use the random grid to search for best hyperparameters
# Create the grid search model and fit to the training data
#svr_grid_search = GridSearchCV(SVR(), param_grid=param_grid, verbose = 3, n_jobs=-1, cv = 3,
                                scoring = 'neg_mean_absolute_error')
#svr_grid_search.fit(x_train, y_train)

In [ ]:
# Use the random grid to search for best hyperparameters
from sklearn.model_selection import RandomizedSearchCV
# Random search of parameters, using 4 fold cross validation, 
# search across 100 different combinations, and use all available cores
svr_random = RandomizedSearchCV(SVR(), param_distributions=param_grid,
                              n_iter = 2, scoring='neg_mean_absolute_error', 
                              cv = 4, verbose=3, n_jobs=-1,
                              return_train_score=True)

# Fit the random search model
svr_random.fit(x_train, y_train)